In [6]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

!pip install openai
!pip install python-dotenv

# Clean up the notebook
clean_notebook()

Notebook cleaned.


In [7]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [8]:
import os
import json
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4.1"    


In [9]:


# Define mock functions
def get_weather(location):
    """Mock weather function - in real scenario, call weather API"""
    weather_data = {
        "Paris, France": "Sunny, 22°C",
        "Bangkok, Thailand": "Cloudy, 28°C", 
        "New York, USA": "Rainy, 18°C",
        "Tokyo, Japan": "Partly cloudy, 25°C"
    }
    return weather_data.get(location, f"Weather data not available for {location}")

def get_time(location):
    """Mock time function - in real scenario, call timezone API"""
    time_data = {
        "Paris, France": "14:30 CET",
        "Bangkok, Thailand": "20:30 ICT",
        "New York, USA": "08:30 EST", 
        "Tokyo, Japan": "22:30 JST"
    }
    return time_data.get(location, f"Time data not available for {location}")

def get_exchange_rate(from_currency, to_currency):
    """Mock exchange rate function"""
    rates = {
        ("USD", "EUR"): 0.85,
        ("USD", "THB"): 33.50,
        ("EUR", "USD"): 1.18,
        ("EUR", "THB"): 39.40
    }
    rate = rates.get((from_currency, to_currency))
    if rate:
        return f"1 {from_currency} = {rate} {to_currency}"
    else:
        return f"Exchange rate not available for {from_currency} to {to_currency}"

# Define tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country e.g. Paris, France"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function", 
        "function": {
            "name": "get_time",
            "description": "Get current time for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country e.g. Paris, France"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_exchange_rate", 
            "description": "Get exchange rate between two currencies",
            "parameters": {
                "type": "object",
                "properties": {
                    "from_currency": {
                        "type": "string",
                        "description": "Source currency code e.g. USD"
                    },
                    "to_currency": {
                        "type": "string", 
                        "description": "Target currency code e.g. EUR"
                    }
                },
                "required": ["from_currency", "to_currency"]
            }
        }
    }
]

# Function mapping
available_functions = {
    "get_weather": get_weather,
    "get_time": get_time, 
    "get_exchange_rate": get_exchange_rate
}



In [10]:
def run_conversation():
    # User query that might trigger multiple function calls
    messages = [
        {
            "role": "user", 
            "content": "I'm planning a trip to Paris and Bangkok. Can you tell me the weather and current time in both cities? Also, what's the exchange rate from USD to EUR and USD to THB?"
        }
    ]
    
    # First API call
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    print("=== Initial Response ===")
    print(f"Content: {response_message.content}")
    print(f"Tool calls: {len(tool_calls) if tool_calls else 0}")
    
    # Check if the model wants to call functions
    if tool_calls:
        # Add the assistant's response to messages
        messages.append(response_message)
        
        # Process each function call
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"\n=== Calling Function: {function_name} ===")
            print(f"Arguments: {function_args}")
            
            # Call the function
            if function_name == "get_exchange_rate":
                function_response = function_to_call(
                    from_currency=function_args.get("from_currency"),
                    to_currency=function_args.get("to_currency")
                )
            else:
                function_response = function_to_call(
                    location=function_args.get("location")
                )
            
            print(f"Response: {function_response}")
            
            # Add function response to messages
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            })
        
        # Second API call to get final response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        
        print("\n=== Final Response ===")
        print(second_response.choices[0].message.content)
        
        return second_response.choices[0].message.content
    else:
        return response_message.content

# Example usage
if __name__ == "__main__":
    # Run the conversation
    final_response = run_conversation()
    
    print("\n" + "="*50)
    print("CONVERSATION COMPLETED")
    print("="*50)

# Additional example: Sequential function calls
def sequential_function_calls():
    """Example showing how to handle sequential function calls"""
    
    messages = [
        {
            "role": "user",
            "content": "First tell me the weather in Paris, then based on that weather, tell me the time there too."
        }
    ]
    
    print("\n=== Sequential Function Calls Example ===")
    
    # This might require multiple rounds of API calls
    max_iterations = 5
    iteration = 0
    
    while iteration < max_iterations:
        iteration += 1
        print(f"\n--- Iteration {iteration} ---")
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        response_message = response.choices[0].message
        
        # Add assistant response to messages
        messages.append(response_message)
        
        # Check if there are tool calls
        if response_message.tool_calls:
            # Process each tool call
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                function_to_call = available_functions[function_name]
                
                print(f"Calling: {function_name}({function_args})")
                
                # Execute function
                if function_name == "get_exchange_rate":
                    function_response = function_to_call(
                        from_currency=function_args.get("from_currency"),
                        to_currency=function_args.get("to_currency")
                    )
                else:
                    function_response = function_to_call(
                        location=function_args.get("location")
                    )
                
                # Add function response to messages
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool", 
                    "name": function_name,
                    "content": function_response
                })
        else:
            # No more tool calls, we have the final response
            print("Final response:", response_message.content)
            break
    
    return messages

# Run sequential example
# sequential_function_calls()

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_api*****here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}